<a href="https://colab.research.google.com/github/Simmus7/IA_Python/blob/main/IA_ConnectFourGame_MIN_MAX_Algorithm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [37]:
import pydot
from IPython.display import Image, display
import queue
import numpy as np
import random as rand

#Clase Node

In [5]:
class Node ():
  def __init__(self, state,value,operators,operator=None, parent=None,objective=None):
    self.state= state
    self.value = value
    self.children = []
    self.parent=parent
    self.operator=operator
    self.objective=objective
    self.level=0
    self.operators=operators
    self.v=0

  def add_child(self, value, state, operator):
    node=type(self)(value=value, state=state, operator=operator,parent=self,operators=self.operators)
    node.level=node.parent.level+1
    self.children.append(node)
    return node
  
  def add_node_child(self, node):
    node.level=node.parent.level+1
    self.children.append(node)    
    return node

  def getchildrens(self):
    return [
        self.getState(i) 
          if not self.repeatStatePath(self.getState(i)) 
            else None for i, op in enumerate(self.operators)]

  def getState(self, index):
    pass
  #Qué son eq y it?
  def __eq__(self, other):
    return self.state == other.state
 
  def __lt__(self, other):
    return self.f() < other.f()
   
  def repeatStatePath(self, state):
      n=self
      while n is not None and n.state!=state:
          n=n.parent
      return n is not None
    
  def pathObjective(self):
      n=self
      result=[]
      while n is not None:
          result.append(n)
          n=n.parent
      return result
  
  def heuristic(self):
    return 0
  
  def cost(self):
    return 1

  def f(self): 
    return self.cost()+self.heuristic()

  def isObjective(self):
    return (self.state==self.objetive.state)


#Clase Tree

In [6]:
class Tree ():
  def __init__(self, root ,operators):
    self.root=root
    self.operators=operators

  def printPath(self,n):
    stack=n.pathObjective()
    path=stack.copy()
    while len(stack)!=0:
        node=stack.pop()
        if node.operator is not None:
            print(f'operador:  {self.operators[node.operator]} \t estado: {node.state}')
        else:
            print(f' {node.state}')
    return path

  def reinitRoot(self):
    self.root.operator=None
    self.root.parent=None
    self.root.objective=None
    self.root.children = []
    self.root.level=0
    
  def miniMax(self, depth):
    self.root.v=self.miniMaxR(self.root, depth, self.root.player)
    ## Comparar los hijos de root
    values=[c.v for c in self.root.children]
    if self.root.player:
      maxvalue=max(values)
      index=values.index(maxvalue)
    else:
      minValue=min(values)
      index=values.index(minValue)

    return self.root.children[index]

  def miniMaxR(self, node, depth, maxPlayer):
    #Condición de parada
    if depth==0 or node.isObjective():
      node.v=node.heuristic()
      return node.heuristic()
    ## Generar los hijos del nodo
    children=node.getchildrens()
    
    ## Según el jugador que sea en el árbol
    if maxPlayer:
      value=float('-inf')
      for i,child in enumerate(children):
        if child is not None:
          newChild=type(self.root)(value=node.value+'-'+str(i),state=child,operator=i,parent=node, 
                                   operators=node.operators,player=False)
          newChild=node.add_node_child(newChild)
          value=max(value,self.miniMaxR(newChild,depth-1,False))
    else:
      value=float('inf')
      for i,child in enumerate(children):
        if child is not None:
          newChild=type(self.root)(value=node.value+'-'+str(i),state=child,operator=i,parent=node,
                                   operators=node.operators,player=True)
          newChild=node.add_node_child(newChild)
          value=min(value,self.miniMaxR(newChild,depth-1,True))
    node.v=value
    return value

  def alpha_beta(self, depth):
    self.root.v= self.alpha_betaR(self.root, depth, float('-inf'), float('+inf'), self.root.player)
    values=[c.v for c in self.root.children]
    if self.root.player:
      maxvalue=max(values)
      index=values.index(maxvalue)
    else:
      minValue=min(values)
      index=values.index(minValue)

    return self.root.children[index]
 
  
  def alpha_betaR(self, node, depth, alpha, beta, player):
    if depth == 0 or node.isObjective():
       node.v = node.heuristic()
       return node.heuristic()
    if player:
      value=float('-inf')
      children = node.getchildrens()
      for i,child in enumerate(children):
        if child is not None:
          newChild=type(self.root)(value=node.value+'-'+str(i),state=child,operator=i,parent=node, operators=node.operators,player=False)
          newChild=node.add_node_child(newChild)
          value = max(value,self.alpha_betaR(newChild, depth-1, alpha,beta,False))
          alpha = max(alpha,value)
          if alpha>=beta:
            break
     
    else:
      value=float('inf')
      children = node.getchildrens()
      for i,child in enumerate(children):
        if child is not None:
          newChild=type(self.root)(value=node.value+'-'+str(i),state=child,operator=i,parent=node, operators=node.operators,player=True)
          newChild=node.add_node_child(newChild)
          value = min(value,self.alpha_betaR(newChild, depth-1, alpha,beta,True))
          beta = min(beta,value)
          if alpha>=beta:
            break
    node.v = value
    return value

  ## Método para dibujar el árbol        
  def draw(self,path):
    graph = pydot.Dot(graph_type='graph')
    nodeGraph=pydot.Node(str(self.root.state)+"-"+str(0),
                          label=str(self.root.state),shape ="circle", 
                          style="filled", fillcolor="red")
    graph.add_node(nodeGraph)
    path.pop()
    return self.drawTreeRec(self.root,nodeGraph,graph,0,path.pop(),path)
  
  ## Método recursivo para dibujar el árbol  
  def drawTreeRec(self,r,rootGraph,graph,i,topPath,path):
    if r is not None:
      children=r.children
      for j,child in enumerate(children):
        i=i+1
        color="white"
        if topPath.value==child.value:
          if len(path)>0:topPath=path.pop()
          color='red'
        c=pydot.Node(child.value,label=str(child.state)+r"\n"+r"\n"+"f="+str(child.heuristic())+r"\n"+str(child.v), 
                      shape ="circle", style="filled", 
                      fillcolor=color)
        graph.add_node(c)
        graph.add_edge(pydot.Edge(rootGraph, c, 
                                  label=str(child.operator)+'('+str(child.cost())+')'))
        graph=self.drawTreeRec(child,c,graph,i,topPath,path)  # recursive call
      return graph
    else:
      return graph  

#Clase NodeConnectFour

In [42]:
class NodeConnectFour(Node):

  def __init__(self, player=True,**kwargs):
    super(NodeConnectFour, self).__init__(**kwargs)
    self.player=player
    if player:
      self.v=float('-inf')
    else:
      self.v=float('inf')

  #Sí es más eficiente reversar para ver las primeras filas primero o es mejor no reversar? la reversa es O(n), creo que no, en fin, luego se cambia
  def getState(self, index):
    state=self.state
    
    #If the column's last position is not empty, the play is ilegal
    if state[0][index] != 0:
      return None
 
    nextState=[r.copy() for r in state]
    #Reverse the state so its more efficient in the first plays (more common and more childrens, more used)
    nextState.reverse()
    lastEmpty = -1
    #We check what is the next possible play in the column (index) given
    for i, row in enumerate(nextState):
      if row [index] == 0:
        lastEmpty = i
        break
    #If lastEmpty wasn't modified that means the column is full of 0s. Then the play can be done in the first available position
    if lastEmpty == -1:
      lastEmpty = 0
    
    nextState[lastEmpty][index] = 1 if self.player else 2
    nextState.reverse()

    return nextState if state!=nextState else None


  def cost(self):
    return self.level
  
  def getGroupsOfFour(self, aux_array):
    g_number = len(aux_array)-3
    inf = 0
    toReturn = []

    for i in range(g_number):
      array_subGroup = []
      for j in range (4):
        array_subGroup.append(aux_array[inf+j])
      inf = inf + 1
      toReturn.append(array_subGroup)
    return toReturn

  #Given a board, returns True if the game is over (someone won or the board is full)
  def isObjective (self):
    board = np.array(self.state)
    board = np.flipud(board)

    #Takes the groups of four of the rows, starting from below (the state was rotated)
    #and checks if there is a winner
    for row in board:
      groups = self.getGroupsOfFour(row)
      for group in groups:
        if self.haveWon(group):
          return True
      
    #Same thing but with columns, transposing the board
    board = board.T
    for col in board:
      groups = self.getGroupsOfFour(col)
      for group in groups:
        if self.haveWon(group):
          return True

    #If nobody has won in columns or in rows, we checks diagonals
    rows, columns = board.shape
    diagonals = []
    number_up_diags = columns - 4
    number_low_diags = rows - 4
    number_total_diags = number_low_diags + number_up_diags + 1

    #Checks main diagoal
    main_diag_groups = self.getGroupsOfFour(np.diag(board, 0))
    for diag in main_diag_groups:
      if self.haveWon(diag):
        return True     
    #Checks upper diagonals
    for i in range (number_up_diags):
      up_diags_groups = self.getGroupsOfFour(np.diag(board, i+1))
      for diag in up_diags_groups:
        if self.haveWon(diag):
          return True

    #Checks upper diagonals
    for i in range (number_low_diags):
      low_diags_groups = self.getGroupsOfFour(np.diag(board, i-1))
      for diag in up_diags_groups:
        if self.haveWon(diag):
          return True

    #If there is no victory, we check if the whole board is full (tie)

    aux_board = board != 0
    if aux_board.all():
      return True

    return False

  #Given an array of 4, checks if the array is full of 1s or 2s
  def haveWon (self, array):
    aux_array = np.array(array)
    allOnes = aux_array == 1
    if allOnes.all():
      return True
    allTwos = aux_array == 2
    if allTwos.all():
      return True
    return False

  def heuristic(self):
    return 0

#Gameplay/Pruebas

In [60]:
b_rows = 6
b_cols = 7

initState = [[0 for i in range(b_cols)] for j in range(b_rows)]

initState[4][0] = 1
initState[3][1] = 1
initState[2][2] = 1
initState[1][3] = 0

initState[5][2] = 1
initState[3][2] = 1
initState[4][2] = 1


initState

[[0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0],
 [0, 0, 1, 0, 0, 0, 0],
 [0, 1, 1, 0, 0, 0, 0],
 [1, 0, 1, 0, 0, 0, 0],
 [0, 0, 1, 0, 0, 0, 0]]

In [49]:
operators = [i for i, f in enumerate(initState)]
operators

[0, 1, 2, 3, 4, 5]

In [54]:
nodeInit=NodeConnectFour(True,value="",state=initState, operators = operators)

nodeInit.isObjective()

False

In [62]:
nodeInit=NodeConnectFour(True,value="",state=initState, operators = operators)

nodeInit.getState(2)

[[0, 0, 0, 0, 0, 0, 0],
 [0, 0, 1, 0, 0, 0, 0],
 [0, 0, 1, 0, 0, 0, 0],
 [0, 1, 1, 0, 0, 0, 0],
 [1, 0, 1, 0, 0, 0, 0],
 [0, 0, 1, 0, 0, 0, 0]]